In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy
import os
import glob
import pandas as pd

PATH = ""

In [2]:
PATH = ""
!ls {PATH}

fastai
labeled_text
models
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
movie-review-sentiment-analysis-kernels-only-update1.ipynb
sampleSubmission.csv
test.tsv
TEXT_imdb.pkl
TEXT.pkl
tmp
train
train.tsv
valid


In [3]:
df_train = pd.read_csv(f'{PATH}train.tsv', sep="\t")

In [4]:
df_test = pd.read_csv(f'{PATH}test.tsv',sep="\t")

In [8]:
df_test.head(10)

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
5,156066,8545,intermittently pleasing but
6,156067,8545,intermittently pleasing
7,156068,8545,intermittently
8,156069,8545,pleasing
9,156070,8545,but


In [6]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


# Language Model

In [7]:
#drop incomplete sentences in training set
def dropIncomplete (df, label="SentenceId", keepLocation=0):
    sentences = []
    for i in df[label].unique():  
        sentences.append(df[df[label] == i].reset_index(drop=True).loc[keepLocation])
    df_result = pd.DataFrame(sentences)
    return df_result

In [9]:
df_train = dropIncomplete(df_train)

In [10]:
df_test = dropIncomplete(df_test)

In [11]:
df_train = pd.DataFrame(df_train["Phrase"]).reset_index(drop=True)
df_test = pd.DataFrame(df_test["Phrase"]).reset_index(drop=True)

In [12]:
df_train.head()

,Phrase
0,A series of escapades demonstrating the adage ...
1,"This quiet , introspective and entertaining in..."
2,"Even fans of Ismail Merchant 's work , I suspe..."
3,A positively thrilling combination of ethnogra...
4,Aggressive self-glorification and a manipulati...


In [13]:
df_test.head()

,Phrase
0,An intermittently pleasing but mostly routine ...
1,Kidman is really the only thing that 's worth ...
2,Once you get into its rhythm ... the movie bec...
3,I kept wishing I was watching a documentary ab...
4,"Kinnear does n't aim for our sympathy , but ra..."


In [14]:
df_full = df_train.append(df_test).reset_index(drop=True)

In [15]:
df_full[df_full.shape[1]-10:]

,Phrase
11830,"At its worst , it implodes in a series of very..."
11831,Maybe I found the proceedings a little bit too...
11832,Moretti 's compelling anatomy of grief and the...
11833,Montias ... pumps a lot of energy into his nic...
11834,Not sweet enough to liven up its predictable s...
11835,"Nasty , ugly , pointless and depressing , even..."
11836,"With tightly organized efficiency , numerous f..."
11837,They should have called it Gutterball .
11838,"A long-winded , predictable scenario ."


In [16]:
df_full.shape

(11839, 1)

In [17]:
spacy_tok = spacy.load('en')

In [18]:
TEXT = data.Field(lower=True, tokenize="spacy")

In [19]:
bs=64; bptt=70

In [20]:
df_train.shape, df_test.shape

((8529, 1), (3310, 1))

In [21]:
md = LanguageModelData.from_dataframes(path=PATH,field=TEXT,col="Phrase",train_df=df_full, val_df=df_test,
                                       test_df=df_test,bs=bs, bptt=bptt)

In [22]:
pickle.dump(TEXT, open(f'{PATH}TEXT_full.pkl','wb'))

In [23]:
ls

fastai@
labeled_text/
models/
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
movie-review-sentiment-analysis-kernels-only-update1.ipynb
sampleSubmission.csv
test.tsv
TEXT_full.pkl
TEXT_imdb.pkl
TEXT.pkl
tmp/
train/
train.tsv
valid/


In [24]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(58, 17902, 1, 268557)

In [25]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', '<', '>', 'eos', '.', 'the', ',', 'a', 'and', 'of', 'to']

In [26]:
TEXT.vocab.stoi['to']

11

In [27]:
em_sz = 200
nh = 500
nl = 3

In [28]:
opt_fn = partial(optim.Adam, betas = (0.7, 0.99))

In [29]:
learner = md.get_model(opt_fn, em_sz, nh, nl, dropouti=0.05, dropout=0.05,
                       wdrop=0.1, dropoute=0.02, dropouth=0.05)

In [30]:
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [31]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                            
    0      6.507587   5.962946  
    1      5.945677   5.586362                            
    2      5.663286   5.501643                            
    3      5.507115   5.266566                            
    4      5.305971   5.084174                            
    5      5.152265   4.990263                            
    6      5.043714   4.964539                            
    7      5.019145   4.864054                            
    8      4.899904   4.71671                             
    9      4.785286   4.603448                            
    10     4.647178   4.474816                            
    11     4.527179   4.389106                            
    12     4.437327   4.326858                            
    13     4.384584   4.287147                            
    14     4.33386    4.274115                            



[array([4.27411])]

In [32]:
learner.save_encoder('adam1_movieReview_full_enc')

In [33]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss                            
    0      4.395538   4.368742  
    1      4.369766   4.235339                            
    2      4.275881   4.11719                             
    3      4.170114   3.993712                            
    4      4.083737   3.91902                             
    5      3.939723   3.773546                            
    6      3.855566   3.699899                            
    7      3.773012   3.633842                            
    8      3.718994   3.601682                            
    9      3.73507    3.595372                            



[array([3.59537])]

In [34]:
learner.save_encoder('adam3_movieReview_full_10_enc')

In [159]:
learner.load_encoder('adam3_movieReview_full_10_enc')

# Test

In [35]:
#use our torchtext field to numericalize it so we can feed it to our language model.
m = learner.model
ss = """This is not something that I will have"""
s = [TEXT.preprocess(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

'this is not something that i will have'

In [36]:
#steps to test a language model
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [37]:
#Top 10 predictions
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['a', 'to', 'been', 'seen', 'made', 'the', 'an', 'watching', ',', 'all']

In [38]:
#generate a bit more text
print(ss,"\n")
for i in range(20):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res, *_ =m(n[0].unsqueeze(0))
print("...")

This is not something that I will have 

a lot of life . < eos > the film is a little too much about the film 's best ...


# Sentiment

In [44]:
#Process training set data with sentiment label
df_trainWLabels = pd.read_csv("train.tsv", sep="\t")
#df_trainWLabels = dropIncomplete(df_trainWLabels)
df_trainWLabels.drop(["PhraseId", "SentenceId"], axis = 1,inplace=True)
df_trainWLabels.reset_index(inplace=True, drop=True)
df_trainWLabels.head()

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,A series of escapades demonstrating the adage ...,2
2,A series,2
3,A,2
4,series,2


In [45]:
len(df_trainWLabels)

156060

In [46]:
df_train_suffled = df_trainWLabels.sample(frac=1)

In [48]:
df_train_suffled.head()

,Phrase,Sentiment
103816,illustrates the ability of the human spirit to...,3
89549,that tells us nothing new,1
134053,conventional arrangements,2
103395,rollicking dark humor,4
25399,Press,2


In [50]:
def IntoLabeledFolders(df, folderPath, validRatio=0.3, contentIndex=0, labelIndex=1):
    count = 1
    L = len(df)
    cutPoint = int(round((1-validRatio)*L))

    for index, row in df.iterrows():
        phrase=row[contentIndex]
        label = row[labelIndex]
        if (count<cutPoint):
            filename = os.path.join(folderPath,"train", str(label), str(count))+".txt"
        else:
            filename = os.path.join(folderPath,"valid", str(label), str(count))+".txt"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename,"w") as f:
            f.write(phrase)
        count = count + 1

In [51]:
IntoLabeledFolders(df=df_train_suffled, folderPath = PATH)

In [52]:
#Show labels ###Start here if already created labeled folders
labelList = !ls {PATH}train
labelList

['0', '1', '2', '3', '4']

In [53]:
#Count number of samples under each label training
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}train/{i}
    labelCount.append(len(trn_files))
labelCount

[4910, 19140, 55597, 23204, 6390]

In [54]:
#Count number of samples under each label in validation
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}valid/{i}
    labelCount.append(len(trn_files))
labelCount

[2162, 8133, 23985, 9723, 2816]

In [55]:
#Review sentences
label = 3
fileIndex = 40
FileList = !ls {PATH}train/{label}
sample = !cat {PATH}train/{label}/{FileList[fileIndex]}
sample

['quite fun in places']

In [56]:
class MultiLabeledMovieReview(torchtext.data.Dataset):
    """Create a MovieReview dataset instance given a path and fields.
    Arguments:
        path: Path to the dataset's highest level directory
        text_field: The field that will be used for text data.
        label_field: The field that will be used for label data.
        Remaining keyword arguments: Passed to the constructor of
            data.Dataset.
    """
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['0','1','2','3','4']:
            fnames = glob.glob(os.path.join(path, label, "*.txt"))
            assert fnames, f"can't find file under {path}/{label}"
            for fname in fnames:
                with open(fname, "r") as f: text = f.readline()
                examples.append(data.Example.fromlist([text,label], fields))
        super().__init__(examples, fields, **kwargs)
    
    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data', 
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [57]:
MovieReviewLabel = data.Field(sequential=False)

In [58]:
TEXT = pickle.load(open(f"{PATH}TEXT_full.pkl","rb")) #open in read and bineary mode

In [59]:
PATH

''

In [60]:
splits = MultiLabeledMovieReview.splits(TEXT, MovieReviewLabel,PATH,train="train", test="valid")

In [61]:
t = splits[0].examples[5]

In [62]:
t.label, " ".join(t.text[:30])

('0', 'worst -- and only -- killer website movie')

In [63]:
bs = 64; bptt = 70

In [64]:
md2 = TextData.from_splits(PATH, splits, bs)

In [65]:
ls

fastai@
labeled_text/
models/
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
movie-review-sentiment-analysis-kernels-only-update1.ipynb
sampleSubmission.csv
test.tsv
TEXT_full.pkl
TEXT_imdb.pkl
TEXT.pkl
tmp/
train/
train.tsv
valid/


In [66]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [67]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [68]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('adam3_movieReview_full_10_enc')

In [69]:
m3.clip=25.
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [70]:
# freeze/unfreeze and differential learning rate
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

epoch      trn_loss   val_loss   accuracy                     
    0      1.239467   1.489791   0.370741  



epoch      trn_loss   val_loss   accuracy                     
    0      1.199971   1.334397   0.422591  



[array([1.3344]), 0.4225905110427902]

In [71]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                     
    0      1.151016   1.312816   0.44679   
    1      1.13149    1.275842   0.456857                     
    2      1.099791   1.261599   0.475013                     
    3      1.077342   1.24544    0.475849                     
    4      1.06892    1.212044   0.489163                     
    5      1.012308   1.215472   0.495478                      
    6      1.039787   1.194805   0.502457                      
    7      0.978599   1.170109   0.51314                       
    8      0.983396   1.177563   0.506015                      
    9      0.982192   1.145562   0.528547                      
    10     0.956515   1.124437   0.531809                      
    11     0.935276   1.101631   0.540259                      
    12     0.938242   1.131697   0.530909                      
    13     0.894443   1.095087   0.548816                      



[array([1.09509]), 0.5488164565028504]

In [72]:
m3.fit(lrs, 3, metrics=[accuracy], cycle_len=10, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                      
    0      0.982388   1.059169   0.554559  
    1      0.92926    1.110146   0.534676                      
    2      0.904218   1.072493   0.558182                      
    3      0.892268   1.060891   0.573155                      
    4      0.878671   1.0254     0.578908                      
    5      0.872884   1.014549   0.585943                      
    6      0.848894   1.029691   0.586342                      
    7      0.838369   1.032747   0.583606                      
    8      0.838001   1.038027   0.581896                      
    9      0.826476   1.017405   0.588137                      
    10     0.865661   1.040834   0.582376                      
    11     0.860403   1.110318   0.536826                      
    12     0.838032   1.058962   0.583938                      
    13     0.834796   1.004079   0.59707                       
    14     0.824735   1.037425   0.574622                   

[array([1.03787]), 0.6033356388409933]

In [73]:
m3.fit(lrs*2, 3, metrics=[accuracy], cycle_len=10, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                      
    0      0.831726   1.030828   0.598953  
    1      0.829601   1.062179   0.586733                      
    2      0.803222   1.052154   0.605893                      
    3      0.789987   1.01909    0.597405                      
    4      0.774232   1.025031   0.607534                      
    5      0.765398   1.010801   0.614761                      
    6      0.729366   1.038643   0.607562                      
    7      0.720756   1.039566   0.610112                      
    8      0.701403   1.022745   0.611431                      
    9      0.705875   1.025065   0.610959                      
    10     0.809913   1.030223   0.598945                      
    11     0.760568   1.079839   0.60264                       
    12     0.762691   1.019638   0.612551                      
    13     0.730351   1.157056   0.593428                      
    14     0.746048   1.020188   0.597844                   

KeyboardInterrupt: 

# Using a transfered model trained on IMDB data set

In [48]:
TEXT = pickle.load(open("TEXT_imdb.pkl","rb")) #TEXT field for IMDB NLP model
splits = MultiLabeledMovieReview.splits(TEXT, MovieReviewLabel, PATH,train="train", test="valid")

In [49]:
bs = 64; bptt = 70
md2 = TextData.from_splits(PATH, splits, bs)
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [50]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('adam3_imdb_10_enc') #NLP model trained on IMDB dataset

In [51]:
m3.clip=25
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [52]:
# freeze/unfreeze and differential learning rate
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

epoch      trn_loss   val_loss   accuracy                     
    0      1.139295   1.263814   0.432896  


epoch      trn_loss   val_loss   accuracy                     
    0      1.070088   1.167292   0.488185  


[array([1.16729]), 0.4881846352137183]

In [53]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                     
    0      1.052536   1.141739   0.509442  
    1      1.03386    1.147596   0.511788                      
    2      1.005644   1.210041   0.488842                      
    3      0.961546   1.168524   0.5204                        
    4      0.96673    1.154298   0.534447                      
    5      0.949491   1.172889   0.531779                      
    6      0.956332   1.20941    0.512285                      
    7      0.932446   1.18231    0.534402                      
    8      0.924049   1.2296     0.52465                       
    9      0.901105   1.169018   0.539088                      
    10     0.905614   1.207651   0.517092                      
    11     0.88481    1.211402   0.545079                      
    12     0.885735   1.226406   0.537763                      
    13     0.887049   1.188989   0.545634                      


[array([1.18899]), 0.5456343977969432]

In [54]:
#Not overfitting yet, continue training
m3.fit(lrs, 3, metrics=[accuracy], cycle_len=10, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                      
    0      0.919168   1.194043   0.536609  
    1      0.892308   1.167078   0.558539                      
    2      0.886449   1.177978   0.542236                      
    3      0.889609   1.19618    0.541768                      
    4      0.852477   1.223079   0.535612                      
    5      0.862832   1.18232    0.541867                      
    6      0.84305    1.166159   0.552757                      
    7      0.832242   1.183578   0.551902                      
    8      0.816114   1.201127   0.555013                      
    9      0.833731   1.172674   0.55678                       
    10     0.854878   1.226053   0.541263                      
    11     0.864414   1.158709   0.535323                      
    12     0.832049   1.222211   0.538389                      
    13     0.820791   1.193392   0.545381                      
    14     0.813213   1.188233   0.530849                   

[array([1.18782]), 0.5568040695153538]